In [1]:
from graph_tool.all import *
from numpy.random import randint
from numpy.random import random
from numpy.random import *
from IPython.display import Image #import to displaz images
from matplotlib import *
from gi.repository import Gtk, Gdk, GdkPixbuf, GObject, GLib
import sys, os, os.path
import time
import numpy as np

#things to be done, figure out all the movement and movement in between states
#add different infection probabilities for vaccination and age group
#if person is vaccinated and not vaccinated


In [40]:
#PRE:  g is graph, distribution is string with country name. distribution must be for 20 age groups, normalized to
#      number of vertices in graph g
#POST: assigns age property randomly to the vertices in g, according to distribution

def add_ages(g,distribution):
    path1 = 'Age_Dists/'
    path2 = 'Dist.csv'
    #generate array containing amount of people per age group
    agedistribution = np.genfromtxt(path1+distribution+path2, delimiter=',')[:,1]
    #visit vertices in random order
    
    vs = list(g.vertices())
    shuffle(vs)
    #we will fill all age groups one by one
    a = 1 #current agegroup
    counter = 0 #counter for how many people have been placed in current group
    
    for v in vs:
        if(a > 20):
            #safety measure
            return
        if counter < agedistribution[a-1]:
            #if current age group hasn't gotten enough people assigned, add another, move on to next vertix
            age[v] = a
            agegrouplists[a-1].append(v)
            counter = counter + 1
        else:
            #current age group is full, move to next age group with at least one person
            a = a + 1
            while(agedistribution[a-1] == 0):
                if(a < 20):
                    a = a + 1
                else:
                    #in case number for last age group is 0
                    return
            age[v] = a
            agegrouplists[a-1].append(v)
            counter = 1

In [52]:
def update_state():

    newly_infected.a = False

    removed.a = False


    # visit the nodes in random order

    vs = list(g.vertices())

    shuffle(vs)

    for v in vs:

        if state[v] == I:

            if random() < r: #r: recovery rate

                state[v] = R
                
            if random() < drlist[age[v]-1]:
            
                state[v] = D
                g.clear_vertex(v)
        
        elif state[v] == S:
            
            if random() < vacc_av[age[v]-1]*vacc_ac[age[v]-1]:
                
                state[v] = V #if vaccine is available and accepted, vaccinate
            
            else:
                ns = list(v.out_neighbors())

                if len(ns) > 0:

                    w = ns[randint(0, len(ns))]  # choose a random neighbor

                    if state[w] == I:

                        if random() < i: #infect with probability

                            state[v] = I

                            newly_infected[v] = True
        
        elif state[v] == V:
            
            ns = list(v.out_neighbors())

            if len(ns) > 0:

                w = ns[randint(0, len(ns))]  # choose a random neighbor

                if state[w] == I:

                    if random() < i_vacc:    #vaccinated people can still become infected, although less likely

                        state[v] = I

                        newly_infected[v] = True

       

        if state[v] == R:

            removed[v] = True
       

    # Filter out the recovered vertices

    g.set_vertex_filter(removed, inverted=True)


    # The following will force the re-drawing of the graph, and issue a

    # re-drawing of the GTK window.

    win.graph.regenerate_surface()
    time.sleep(0.5)
    win.graph.queue_draw()


    # if doing an offscreen animation, dump frame to disk

    if offscreen:

        global count

        pixbuf = win.get_pixbuf()

        pixbuf.savev(r'./frames/sirs%06d.png' % count, 'png', [], [])

        if count > max_count:

            sys.exit(0)

        count += 1


    # We need to return True so that the main loop will call this function more

    # than once.

    return True


In [51]:
# SET UP GRAPH

g = Graph(directed=False)

#definition of vertex properties
S = [1, 1, 1, 1]           # White color

I = [0, 0, 0, 1]           # Black color

R = [0.5, 0.5, 0.5, 1.]    # Grey color (will not actually be drawn)

V = [0, 0, 1, 1]           # Blue color

D = [0.8, 0, 0, 0.6]       #DEATH (red)

#all definitions of vertice edges
state = g.new_vertex_property("vector<double>")
age=g.new_vertex_property("int")
vac=g.new_vertex_property("bool")
removed = g.new_vertex_property("bool")
newly_infected = g.new_vertex_property("bool")

agegroups=20
agegrouplists = [[] for i in range(1, agegroups+1)]

#probabilities
i = 0.8  # S->I probability

i_vacc = 0.2 #V->I probability

#death probability list for different age groups
drlist = [0,0,0,0,0,0,0,0,0,0,0,0,.10,.20,.30,.40,.50,.60,.90,1]

#vaccine availability by age group:
vacc_av = [0,0,0,0,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625]
#vaccine acceptance rate by age group:
vacc_ac = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

#insert random vertices (nodes)
g.add_vertex(100)

# insert some random links
for s,t in zip(randint(0, 100, 100), randint(0, 100, 100)):
    g.add_edge(g.vertex(s), g.vertex(t))
# layout positions
pos = sfdp_layout(g)


#makes all nodes suceptible
for i in g.vertices():
    v = g.vertex(i) 
    state[v] = S

#this line is to randomlyinitialize one vertex as infected
state[randint(0,100)]= I

#assign ages to vertices following given age distribution
add_ages(g,'Niger')

#current placeholder for adding vaccination properties
if 1 == 2:
    for i in g.vertices():
        v = g.vertex(i)
        vac[v] = False

In [39]:
for i in range(1,agegroups+1):
    print(i,': ',len(agegrouplists[i-1]))

1 :  20
2 :  16
3 :  14
4 :  11
5 :  9
6 :  7
7 :  5
8 :  4
9 :  3
10 :  3
11 :  2
12 :  2
13 :  2
14 :  1
15 :  1
16 :  0
17 :  0
18 :  0
19 :  0
20 :  0


In [4]:
ages = np.genfromtxt('Age_Dists/JapanDist.csv', delimiter=',')[:,1]
print(ages)

[4. 4. 4. 4. 5. 5. 5. 6. 7. 8. 7. 6. 6. 6. 7. 6. 4. 3. 2. 0.]


In [53]:
#DISPLAY SETTINGS

#Displaying how the graph will be initialized
graph_draw(g, vertex_text=g.vertex_index, output="two-nodes.png")
fig = Image(filename=('two-nodes.png'))
fig


# If True, the frames will be dumped to disk as images.
# CAN SOMONE FIGURE OUT WHAT THIS DOES???????????????????????????????

offscreen = sys.argv[1] == "offscreen" if len(sys.argv) > 1 else False

max_count = 500

if offscreen and not os.path.exists("./frames"):

    os.mkdir("./frames")
#?????????????????????????????????????????????????????????????????????


# This creates a GTK+ window with the initial graph layout,basically creates the pop up window

if not offscreen:

    win = GraphWindow(g, pos, geometry=(500, 400),

                      edge_color=[0.6, 0.6, 0.6, 1],

                      vertex_fill_color=state,

                      vertex_halo=newly_infected,

                      vertex_halo_color=[0.8, 0, 0, 0.6])

else:

    count = 0

    win = Gtk.OffscreenWindow()

    win.set_default_size(500, 400)

    win.graph = GraphWidget(g, pos,

                            edge_color=[0.6, 0.6, 0.6, 1],

                            vertex_fill_color=state,

                            vertex_halo=newly_infected,

                            vertex_halo_color=[0.8, 0, 0, 0.6])

    win.add(win.graph)

In [42]:
drlist[age[10]]

0

In [54]:
cid = GLib.idle_add(update_state)


# We will give the user the ability to stop the program by closing the window.

win.connect("delete_event", Gtk.main_quit)


# Actually show the window, and start the main loop.

win.show_all()

Gtk.main()

In [11]:
g.list_properties()

In [8]:
g.save("my_graph.xml.gz")
g2 = load_graph("my_graph.xml.gz")